In [37]:
!pip install chromedriver-autoinstaller
!pip install wapy

  Preparing metadata (setup.py) ... done
  Created wheel for wapy: filename=wapy-1.0.0-py3-none-any.whl size=7659 sha256=38b9cd5c7c1a844de9b5d7133515af75d9d78053cde344a79946a9cc720d8e82
  Stored in directory: /Users/yash/Library/Caches/pip/wheels/8a/31/22/33eb0845bcc356e76155752438dc10ea118c25682a078ab9ea
Successfully built wapy


In [6]:



chromedriver_autoinstaller.install()

CHROME >= 115.0.5763.0, using mac-x64 as architecture identifier


'/Users/yash/anaconda3/lib/python3.11/site-packages/chromedriver_autoinstaller/119/chromedriver'

In [431]:
import requests
import os
import random
import time
import json

import pandas as pd

from tqdm import tqdm

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import chromedriver_autoinstaller
import chromedriver_binary



In [432]:


def open_browser():
    """
    Opens a new automated browser window with all tell-tales of automated browser disabled
    """
    options = webdriver.ChromeOptions()
    options.add_argument("start-maximized")
    
    # remove all signs of this being an automated browser
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)

    # open the browser with the new options
    driver = webdriver.Chrome(options=options)
    return driver

In [502]:
driver = open_browser()

In [434]:
def load_walmart(driver, attempts = 5):
    
    for attempt in range(0, attempts):
        # load the website
        driver.get('https://www.walmart.com')

        # make sure we wait with searching until the actual search bar appeared
        wait = 10
        try:
            search_box = WebDriverWait(driver, wait).until(EC.presence_of_element_located((By.CLASS_NAME, 'search-bar')))
            # no exception is thrown which means the wait is over and we found the search bar. We can finish this function
            return 
        except:
            # Since we're here, the search bar was not found. We just go back to the top of the for loop again
            continue

In [503]:
load_walmart(driver)

In [369]:
def search(query, driver):
    search_box = driver.find_element(By.CLASS_NAME, 'search-bar')
    for i in range(100):
        search_box.send_keys(Keys.BACKSPACE)
    search_box.send_keys(query + Keys.ENTER)

In [357]:
def get_suggestions(query):
    headers = {
        'authority': 'www.walmart.com',
        'accept': 'application/json',
        'accept-language': 'en-US',
        'device_profile_ref_id': 'LqIurHOiqfXM8Htfp_F9D8H7SeOsKpC8j3tj',
        'downlink': '10',
        'dpr': '2',
        'referer': 'https://www.walmart.com/',
        'sec-ch-ua': '"Google Chrome";v="119", "Chromium";v="119", "Not?A_Brand";v="24"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'traceparent': '00-b0ceab7f1e19f66d6cf2aa398fbd522a-95e65dbeedb13e57-00',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
        'wm_mp': 'true',
        'wm_qos.correlation_id': '7IMIWhZT7hpkZp3fl8paHlrYnJqgGqRXYLcP',
        'x-enable-server-timing': '1',
        'x-latency-trace': '1',
        'x-o-bu': 'WALMART-US',
        'x-o-correlation-id': '7IMIWhZT7hpkZp3fl8paHlrYnJqgGqRXYLcP',
        'x-o-mart': 'B2C',
        'x-o-platform': 'rweb',
        'x-o-platform-version': 'us-web-1.104.1-fe2ff4adf0765367f55d20cf86cfa7fcc85ec064-1101',
        'x-o-segment': 'oaoh',
    }

    params = {
        'term': query,
        'num': '9',
    }

    response = requests.get('https://www.walmart.com/typeahead/v3/complete', params=params, cookies={}, headers=headers)
    response = response.json()
    suggestions = response['queries']
    processed_suggestions = []
    
    for suggestion in suggestions:
        processed_suggestions.append(suggestion['displayName'])
        
    return processed_suggestions

In [108]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'
queries = []
for letter in tqdm(alphabet):
    suggestions = get_suggestions(letter)
    queries += suggestions
    time.sleep(2)


In [612]:
selected_queries = queries[15:18]
for query in selected_queries:
    search(query, driver)
    soup = BeautifulSoup(driver.page_source)
    soup = soup.find(id="__next")
    products = soup.find_all('div', attrs={'class': 'mb0 ph1 pa0-xl bb b--near-white w-25'})
    i = 0
    for product in products:
        
        try:
            if product:    
                name = product.find('span',attrs={'data-automation-id': 'product-title'}).text
                price = product.find('div',attrs={'data-automation-id': 'product-price'}).find('span', attrs={'class':'w_iUH7'}).text
                price = price[price.find('$'):]
                review_stats = product.find('div', attrs={'class': 'flex', 'class':'mt2'}).find('span', attrs={'class':'w_iUH7'})
                stars = review_stats[:review_stats.find(' out')]
                review_count = review_stats[review_stats.find("s. ")+3:]
                sponsored = 'Sponsored' in product.text

                productData = {}
                productData['name'] = name
                productData['price'] = price
                productData['stars'] = stars
                productData['reviews'] = review_count
                productData['sponsored'] = sponsored
                productData['rank'] = i
                all_prod.append(productData)
                i += 1
        except Exception as e:
            print(e)
        

'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'


In [611]:
len(all_prod)

530

In [365]:
pd.DataFrame(all_prods).drop_duplicates(subset=['name', 'price', 'stars', 'reviews'])

""


In [613]:
all_prod

[{'name': 'Fresh Cosmic Apples, 3 lb Bag',
  'price': '$6.56',
  'stars': '<span class="w_iUH7">2.5',
  'reviews': '165 reviews</span>',
  'sponsored': True},
 {'name': 'Fresh Envy Apples, Each',
  'price': '$1.48',
  'stars': '<span class="w_iUH7">3.3',
  'reviews': '448 reviews</span>',
  'sponsored': True},
 {'name': 'Cosmic Crisp Apples, Each',
  'price': '$1.35',
  'stars': '<span class="w_iUH7">2.8',
  'reviews': '276 reviews</span>',
  'sponsored': True},
 {'name': 'Opal Apples, 2 lb bag',
  'price': '$4.12',
  'stars': '<span class="w_iUH7">3.8',
  'reviews': '69 reviews</span>',
  'sponsored': True},
 {'name': 'Fresh Gala Apples, 3 lb Bag',
  'price': '$3.46',
  'stars': 'Non',
  'reviews': 'ne',
  'sponsored': False},
 {'name': 'Fresh Gala Apple, Each',
  'price': '$0.64',
  'stars': 'Non',
  'reviews': 'ne',
  'sponsored': False},
 {'name': 'Fresh Honeycrisp Apples, 3 lb Bag',
  'price': '$5.46',
  'stars': 'Non',
  'reviews': 'ne',
  'sponsored': False},
 {'name': 'Fresh Ho